In [1]:
import os
import math
from dataclasses import dataclass
from typing import Tuple, List

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoConfig, AutoModel
from sklearn.preprocessing import MinMaxScaler
import joblib

import matplotlib.pyplot as plt
from IPython import display
from matplotlib.ticker import MaxNLocator
from scipy.interpolate import CubicSpline

import random

In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

In [3]:
SEED = 42
set_seed(SEED)

## Parameter

In [22]:
class Config:
    # I, V, t column names in your exce
    INPUT_COLS = ["Current(A)", "Voltage(V)", "Time(s)"]
    TARGET_COLS = ["Voltage(V)", "SoC(CCM)"]

    OCV_FILE_PATH = '../Data/OCV20.xlsx'
    TRAIN_FILE_PATH = '../Data/20_FUDS.xlsx'
    Model_SAVE_DIR = "../Models/"

    SEQ_LEN = 6000
    TRAIN_SPLIT = 0.8

    # head dims
    INPUT_DIM = 3
    HEAD_DIM = 1

    EPOCHS = 3000
    BATCH_SIZE = 100
    LR = 1e-4
    WEIGHT_DECAY = 0.0

    # battery constants
    ETA = 1.08 # coulombic efficiency
    Q_Ah = 1.1 # rated capacity [Ah]

    # loss weights (paper: α=0.2 for voltage; β=1.0 for SOC dynamics)
    ALPHA = 0.2
    BETA = 1.0

cfg = Config()

In [8]:
cfg.SAVED_MODEL_PATH = os.path.join(cfg.Model_SAVE_DIR, 'Transformer_20_FUDS_model.pth')
cfg.SAVED_SCALER_PATH = os.path.join(cfg.Model_SAVE_DIR, '20_FUDS_scaler.joblib')

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Load Data

In [10]:
class BatteryDataset(Dataset):

    def __init__(self, df: pd.DataFrame, seq_len: int, scaler: MinMaxScaler, config: "Config"):

        self.seq_len = seq_len
        self.scaler = scaler

        # 1. Clean the data
        df_clean = df.dropna(subset=[*config.INPUT_COLS, *config.TARGET_COLS])
        df_clean = df_clean[(df_clean[config.TARGET_COLS[1]] > 0.0) & (df_clean[config.TARGET_COLS[1]] < 1.0)]

        # 2. Extract and convert data to Tensor
        inputs_raw = df_clean.loc[:, config.INPUT_COLS].to_numpy(np.float32)
        model_inputs = df_clean.loc[:, config.INPUT_COLS].to_numpy(np.float32)
        targets = df_clean.loc[:, config.TARGET_COLS].to_numpy(np.float32)
        inputs_scaled = self.scaler.transform(model_inputs)

        self.scaled_data = torch.from_numpy(inputs_scaled)
        self.raw_data = torch.from_numpy(inputs_raw)
        self.target_data = torch.from_numpy(targets)

        n_samples = len(df_clean)

        #in my master thesis experiment, there is only one long sequence
        if n_samples >= self.seq_len:
            # Mode 1: Sequence length is sufficient, perform non-overlapping chunking and discard the remainder
            self.num_sequences = n_samples // self.seq_len
            self._mode = 'chunking'  # Set mode to "chunking"
        else:
            # Mode 2: Sequence length is insufficient, use the entire dataset as a single sequence
            self.num_sequences = 1 if n_samples > 0 else 0 # If empty after cleaning, number of sequences is 0
            self._mode = 'full'      # Set mode to "full"


    def __len__(self):
        return self.num_sequences

    def __getitem__(self, idx: int):

        if self._mode == 'chunking':
            start_idx = idx * self.seq_len
            end_idx = start_idx + self.seq_len

            scaled_inputs = self.scaled_data[start_idx:end_idx]
            raw_inputs = self.raw_data[start_idx:end_idx]
            targets = self.target_data[start_idx:end_idx]

        else: # self._mode == 'full'
            scaled_inputs = self.scaled_data
            raw_inputs = self.raw_data
            targets = self.target_data

        return scaled_inputs, raw_inputs, targets

In [23]:
full_df = pd.read_excel(cfg.TRAIN_FILE_PATH)

current_col = cfg.INPUT_COLS[0]  # "Current(A)"
full_df[current_col] = full_df[current_col] * -1 #Discharge is POSITIVE in paper!!! in Dataset usually negative 

full_df = full_df[(full_df[cfg.TARGET_COLS[1]] > 0.0) & (full_df[cfg.TARGET_COLS[1]] < 1.0)]
split_idx = int(len(full_df) * cfg.TRAIN_SPLIT)

train_df = full_df.iloc[:split_idx]
val_df = full_df.iloc[split_idx:]

print(f"Data split: {len(train_df)} for training, {len(val_df)} for validation.")

Data split: 6372 for training, 1594 for validation.


In [24]:
# --- Fitting Scale and save Scaler ---
scaler = MinMaxScaler() 
scaler.fit(train_df[cfg.INPUT_COLS])
#joblib.dump(scaler, cfg.SAVED_SCALER_PATH)
#print(f"Scaler fitted and saved to {cfg.SAVED_SCALER_PATH}")

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [25]:
sequence_len = len(train_df)

train_dataset = BatteryDataset(train_df, sequence_len, scaler, cfg)
train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE, shuffle=False)

val_dataset = BatteryDataset(val_df, sequence_len, scaler, cfg)
val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE, shuffle=False)

C:\Users\Marven\.conda\envs\PINN\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Marven\.conda\envs\PINN\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


## Model Structure

In [18]:
transformer_config = AutoConfig.from_pretrained(
    "bert-base-uncased", # Use a standard config as a template
    # Key change: Ensure the model can handle our long sequence length
    max_position_embeddings=9000,

    # Increase model complexity for better performance on long sequences
    hidden_size=128,              # Model dimension
    num_hidden_layers=8,          # Number of encoder layers
    num_attention_heads=8,        # Number of attention heads
    intermediate_size=512         # Dimension of the FFN layer
)

# Initialize a model with this new configuration (random weights)
transformer_backbone = AutoModel.from_config(config=transformer_config)

In [14]:
class Transformer(nn.Module):
    
    def __init__(self, backbone: nn.Module, in_dim: int):
        super().__init__()
        self.d_model = backbone.config.hidden_size
        self.in_proj = nn.Linear(in_dim, self.d_model)
        self.backbone = backbone
        self.head = nn.Sequential(
            nn.Linear(self.d_model, 1),
            #nn.Sigmoid()
        )


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.in_proj(x)
        out = self.backbone(inputs_embeds=x).last_hidden_state
        soc = self.head(out)
        return soc.squeeze(-1) # Shape: (B, L)

## Training

In [15]:
# Inject this new backbone into our PITransformer structure
model = Transformer(
    backbone=transformer_backbone,
    in_dim=cfg.INPUT_DIM,
    #out_dim=cfg.HEAD_DIM
).to(device)


In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=cfg.LR, weight_decay=cfg.WEIGHT_DECAY)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
mse = nn.MSELoss()

train_rmse_history = []
train_mae_history = []
val_rmse_history = []
val_mae_history = []
best_val_rmse = 1000


for epoch in range(cfg.EPOCHS):
  model.train()
  total_train_loss, soc_preds_T, soc_trues_T, R0_list, R1_list, C1_list = 0, [], [], [], [], []
  for xs_scaled, xs_raw, ys in train_loader:
    xs_scaled, xs_raw, ys = xs_scaled.to(device), xs_raw.to(device), ys.to(device)
    optimizer.zero_grad()
    soc_true = ys[..., 1]
    pred = model(xs_scaled)
    #loss, loss_ul, loss_uoc, loss_soc= loss_fn(pred, xs_raw, ys)
    loss = mse(pred, soc_true)
    loss.backward()
    optimizer.step()
    total_train_loss += loss.item()


    soc_preds_T.append(pred.detach().cpu())
    soc_trues_T.append(ys[..., 1].cpu())


  avg_train_loss = total_train_loss / len(train_loader)

  all_preds_T = torch.cat(soc_preds_T).view(-1)
  all_trues_T = torch.cat(soc_trues_T).view(-1)

  all_soc_preds_T = all_preds_T.numpy()
  all_soc_trues_T = all_trues_T.numpy()

  soc_rmse_T = np.sqrt(np.mean((all_soc_preds_T - all_soc_trues_T) ** 2))
  soc_mae_T = np.mean(np.abs(all_soc_preds_T - all_soc_trues_T))

  #scheduler.step()

  model.eval()
  total_val_loss, soc_preds, soc_trues = 0.0, [], []
  all_soc_preds, all_soc_trues, all_preds, all_trues = None, None, None, None 


  with torch.no_grad():
      if len(val_loader) > 0:
          for xs_scaled, xs_raw, ys in val_loader:
              xs_scaled, xs_raw, ys = xs_scaled.to(device), xs_raw.to(device), ys.to(device)
              soc_true = ys[..., 1]
              pred = model(xs_scaled)
              loss = mse(pred, soc_true) 
              total_val_loss += loss.item()
              soc_preds.append(pred.detach().cpu())
              soc_trues.append(ys[..., 1].cpu())

          avg_val_loss = total_val_loss / len(val_loader)
          all_preds = torch.cat(soc_preds).view(-1)
          all_trues = torch.cat(soc_trues).view(-1)

          all_soc_preds = all_preds.numpy()
          all_soc_trues = all_trues.numpy()

          soc_rmse = np.sqrt(np.mean((all_soc_preds - all_soc_trues) ** 2))
          soc_mae = np.mean(np.abs(all_soc_preds - all_soc_trues))

          if soc_rmse < best_val_rmse:
            best_val_rmse = soc_rmse
            best_model_epoch = epoch+1
            torch.save(model.state_dict(), cfg.SAVED_MODEL_PATH)
      else:
          avg_val_loss, soc_rmse, soc_mae= 0.0, 0.0, 0.0

  train_rmse_history.append(soc_rmse_T)
  train_mae_history.append(soc_mae_T)
  val_rmse_history.append(soc_rmse)
  val_mae_history.append(soc_mae)

  if epoch % 10 == 0:

    print(f"Epoch {epoch + 1}/{cfg.EPOCHS} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f} | SOC RMSE_T: {soc_rmse_T:.4f} | SOC MAE_T: {soc_mae_T:.4f} | SOC RMSE: {soc_rmse:.4f} | SOC MAE: {soc_mae:.4f}")


In [ ]:
fig_loss, (ax_rmse, ax_mae) = plt.subplots(1, 2, figsize=(16, 6))
epochs_range = range(1, cfg.EPOCHS + 1)

# RMSE Loss curve
ax_rmse.plot(epochs_range, train_rmse_history, label='Train RMSE', color='royalblue', linestyle='-')
ax_rmse.plot(epochs_range, val_rmse_history, label='Validation RMSE', color='darkorange', linestyle='-')
ax_rmse.set_title('RMSE Loss Over Epochs', fontsize=16)
ax_rmse.set_xlabel('Epoch', fontsize=12)
ax_rmse.set_ylabel('RMSE', fontsize=12)
ax_rmse.legend(fontsize=10)
ax_rmse.grid(True)

# MAE Loss curve
ax_mae.plot(epochs_range, train_mae_history, label='Train MAE', color='royalblue', linestyle='-')
ax_mae.plot(epochs_range, val_mae_history, label='Validation MAE', color='darkorange', linestyle='-')
ax_mae.set_title('MAE Loss Over Epochs', fontsize=16)
ax_mae.set_xlabel('Epoch', fontsize=12)
ax_mae.set_ylabel('MAE', fontsize=12)
ax_mae.legend(fontsize=10)
ax_mae.grid(True)

plt.tight_layout()

plt.savefig(cfg.LOSS_CURVE_PATH)
plt.show()